This code reads the tdms files and creates a .csv/excel table containing the information relative to the file, like name, duration etc and which channels are present. Just run all the cells adjusting the paths accordingly to where the data is stored or where you want to save it. Further information is written in the code.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tqdm import tqdm
import os
import lib_bbmdev
import logging
from datetime import datetime
import sys
from pathlib import Path
from nptdms import TdmsFile
import pickle

%matplotlib inline

In [2]:
def tdms_load_make_table(file_name, data_dict=None, print=True):

    tdmsname=file_name.split(sep='/')[-1]
    name= tdmsname.split(sep='_')[0]
    number = name.split(sep='s')[1]
    tdms_file = TdmsFile(file_name)
    group = tdms_file.groups()[0]
    data_lab = []
    duration = len(group.channels()[0])/1000
    count = 0
    for channel in group.channels():
        channel_name = channel.name
        #print("channel:", channel_name, "index:", count)
        count += 1
        if channel_name in data_dict.keys():
            data_lab.append(data_dict[channel_name])
    selected_arr=np.ones(len(group.channels()))
    selected_read_data = []
    for j in range(0, len(group.channels())):
        if selected_arr[j] == 1:
            selected_read_data.append(group.channels()[j])
        else:
            # nanimo sinai
            pass
    # for i in tqdm(range(0, len(selected_read_data)), desc="Data loading"):
    indices = [tdmsname]
    for elem in data_dict.values():
        if elem in data_lab:
            indices.append(1)
        else:
            indices.append(0)
    indices.append(duration)
    indices.append(1000)
    if print:
        print(indices)
    return indices

In [4]:
data_dict = {'cDAQ1Mod1/ai0':'Iu','cDAQ1Mod1/ai1':'Iv',  'cDAQ1Mod1/ai2':'Iw', 
             'cDAQ1Mod2/ai0':'A', 'cDAQ1Mod2/ai1':'B', 'cDAQ1Mod2/ai2':'Z', 'cDAQ1Mod2/ai3':'H',
             'cDAQ1Mod3/ai0':"SV1", 'cDAQ1Mod3/ai1':"SV2", 'cDAQ1Mod3/ai2':"SV3",'cDAQ1Mod3/ai3':"Count",
             'cDAQ1Mod4/ai0':"Vu", 'cDAQ1Mod4/ai1':"Vv", 'cDAQ1Mod4/ai2':'Vw'}

In [16]:
labels = list(data_dict.values())
labels.insert(0,'File')
labels.append('Duration [s]')
labels.append('Sampling Frequency [Hz]')
labels

['File',
 'Iu',
 'Iv',
 'Iw',
 'A',
 'B',
 'Z',
 'H',
 'SV1',
 'SV2',
 'SV3',
 'Count',
 'Vu',
 'Vv',
 'Vw',
 'Duration [s]',
 'Sampling Frequency [Hz]']

In [28]:
os.chdir(r'C:\Users\ggora\DevelopingToolkit\bbm_run2_tdms')
df_signals = {}
path = r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/'


#this cell creates a dictionary of dataframe each one corresponding to a tdms file (very computationally expensive) that you can save in pickle format (approx 20 GB lol)
#anyway I think it's better to open each file singularly without creating this huge database
for file in os.listdir(path):
    print(f'Processing file {file}...')
    name= file.split(sep='_')[0]
    number = name.split(sep='s')[1]
    df_signal = lib_bbmdev.tdms_load_selected_custom(path + file, data_dict, samplerate=1000, print=True)
    df_signals[number]=df_signal
    print(f'File {file} processed.')

Processing file tdms100_room-to-cryo.tdms...


ValueError: Shape of passed values is (12, 913000), indices imply (1, 913000)

In [18]:
os.chdir(r'C:\Users\ggora\DevelopingToolkit\bbm_run2_tdms')
path = r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/tdsm/'
df=[]
df.append(labels)
for file in os.listdir(path):
    print(f'Processing file {file}...')
    name= file.split(sep='_')[0]
    number = name.split(sep='s')[1]
    tdms_indices = tdms_load_make_table(path + file, data_dict, print_=True)
    df.append(tdms_indices)
    print(f'File {file} processed.')

Processing file tdms100_room-to-cryo.tdms...
['tdms100_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 913.0, 1000]
File tdms100_room-to-cryo.tdms processed.
Processing file tdms101_room-to-cryo.tdms...
['tdms101_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 3098.0, 1000]
File tdms101_room-to-cryo.tdms processed.
Processing file tdms102_room-to-cryo.tdms...
['tdms102_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1379.0, 1000]
File tdms102_room-to-cryo.tdms processed.
Processing file tdms103_room-to-cryo.tdms...
['tdms103_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 9348.0, 1000]
File tdms103_room-to-cryo.tdms processed.
Processing file tdms104_room-to-cryo.tdms...
['tdms104_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 15217.0, 1000]
File tdms104_room-to-cryo.tdms processed.
Processing file tdms34_room-to-cryo.tdms...
['tdms34_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 425.0, 1000]
File td

[nptdms.reader WARNING] Last segment of file has unknown size, will attempt to read to the end of the file
[nptdms.tdms_segment WARNING] Data size 608832269 is not a multiple of the chunk size 28000. Will attempt to read last chunk


['tdms63_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 336.0, 1000]
File tdms63_room-to-cryo.tdms processed.
Processing file tdms64_room-to-cryo.tdms...


[nptdms.base_segment WARNING] Cropping data from 269 to 252 bytes to match segment size derived from channels


['tdms64_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 21744.009, 1000]
File tdms64_room-to-cryo.tdms processed.
Processing file tdms66_room-to-cryo.tdms...
['tdms66_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 9186.0, 1000]
File tdms66_room-to-cryo.tdms processed.
Processing file tdms67_room-to-cryo.tdms...
['tdms67_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 868.0, 1000]
File tdms67_room-to-cryo.tdms processed.
Processing file tdms69_room-to-cryo.tdms...
['tdms69_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2396.0, 1000]
File tdms69_room-to-cryo.tdms processed.
Processing file tdms72_room-to-cryo.tdms...
['tdms72_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 16.0, 1000]
File tdms72_room-to-cryo.tdms processed.
Processing file tdms73_room-to-cryo.tdms...
['tdms73_room-to-cryo.tdms', 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2532.0, 1000]
File tdms73_room-to-cryo.tdms processed.
Processing file tdms75_r

In [19]:
tdsm_dataframe = pd.DataFrame(df)
tdsm_dataframe

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,File,Iu,Iv,Iw,A,B,Z,H,SV1,SV2,SV3,Count,Vu,Vv,Vw,Duration [s],Sampling Frequency [Hz]
1,tdms100_room-to-cryo.tdms,1,1,1,1,1,1,1,0,0,0,1,1,1,1,913.0,1000
2,tdms101_room-to-cryo.tdms,1,1,1,1,1,1,1,0,0,0,1,1,1,1,3098.0,1000
3,tdms102_room-to-cryo.tdms,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1379.0,1000
4,tdms103_room-to-cryo.tdms,1,1,1,1,1,1,1,0,0,0,1,1,1,1,9348.0,1000
5,tdms104_room-to-cryo.tdms,1,1,1,1,1,1,1,0,0,0,1,1,1,1,15217.0,1000
6,tdms34_room-to-cryo.tdms,1,1,1,1,1,1,1,1,1,1,0,1,1,1,425.0,1000
7,tdms35_room-to-cryo.tdms,1,1,1,1,1,1,1,1,1,1,0,1,1,1,9959.0,1000
8,tdms46_room-to-cryo.tdms,1,1,1,1,1,1,1,1,1,1,0,1,1,1,277.0,1000
9,tdms47_room-to-cryo.tdms,1,1,1,1,1,1,1,1,1,1,0,1,1,1,429.0,1000


In [20]:
tdsm_dataframe.to_csv(r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/tdsm_dataframe.csv')
tdsm_dataframe.to_excel(r'C:/Users/ggora/DevelopingToolkit/bbm_run2_tdms/Data/2023_run2/tdsm_dataframe.xlsx', engine='xlsxwriter')